In [1]:


import pandas as pd
from datetime import datetime
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.dialects.postgresql import insert


from models import Team
import date_utils as date_mng
from data_manager import DataManager
data_manager = DataManager()
self = data_manager

In [2]:
players = self.pull_players()

An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
An error occurred: HTTPSConnectionPool(host='s

In [ ]:
    @session_management
    def update_player_record(self, session, player):
        insert_stmt = insert(Player).values(
            nba_team_id = team["id"],
            nickname = team["nickname"],
            city = team["city"],
            state = team["state"],
            full_name = team["full_name"],
            abbreviation = team["abbreviation"],
            )

        do_update_stmt = insert_stmt.on_conflict_do_update(
            index_elements=['nba_team_id'], # possible conflicting column
            set_=dict(
                nickname=insert_stmt.excluded.nickname,
                city=insert_stmt.excluded.city,
                state=insert_stmt.excluded.state,
                full_name=insert_stmt.excluded.full_name,
                abbreviation=insert_stmt.excluded.abbreviation
            )
        )

        session.execute(do_update_stmt)
        session.commit()


In [ ]:
@session_management
def update_team_record(self, session, team):
    insert_stmt = insert(Team).values(
        nba_team_id = team["id"],
        nickname = team["nickname"],
        city = team["city"],
        state = team["state"],
        full_name = team["full_name"],
        abbreviation = team["abbreviation"],
    )

    do_update_stmt = insert_stmt.on_conflict_do_update(
        index_elements=['nba_team_id'], # possible conflicting column
        set_=dict(
            nickname=insert_stmt.excluded.nickname,
            city=insert_stmt.excluded.city,
            state=insert_stmt.excluded.state,
            full_name=insert_stmt.excluded.full_name,
            abbreviation=insert_stmt.excluded.abbreviation
        )
    )

    session.execute(do_update_stmt)
    session.commit()